<a href="https://colab.research.google.com/github/kimjaehwankimjaehwan/python_/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. ResNet50 모델 로드:

ResNet50 모델은 사전 학습된 가중치(weights='imagenet')로 로드됩니다. 이 모델은 ImageNet 데이터셋에서 학습된 모델로, 1000개의 클래스를 예측할 수 있습니다.

2. 이미지 로드:

sklearn에서 제공하는 샘플 이미지를 사용합니다. load_sample_image('china.jpg') 또는 load_sample_image('flower.jpg')로 이미지를 로드할 수 있습니다.

3. 이미지 전처리:

이미지를 ResNet50 모델에 맞는 입력 크기인 224x224로 조정합니다. 이때 이미지 배열을 numpy.resize를 사용해 조정합니다.
전처리 함수 preprocess_input을 사용하여 이미지 데이터를 ResNet50이 기대하는 형식으로 변환합니다. 이 함수는 픽셀 값을 적절한 범위로 조정합니다.

4. 예측 수행:

model.predict(image)를 사용하여 예측을 수행합니다. 이 모델은 입력된 이미지에 대한 예측 확률을 반환합니다.

5. 예측 결과 디코딩:

decode_predictions를 사용하여 예측 결과를 해석 가능한 레이블로 디코딩합니다. top=3을 사용하여 가장 가능성이 높은 3개의 클래스를 출력합니다.

6. 결과 출력:

각 예측 결과에 대해 레이블과 해당 확률을 출력합니다.

In [1]:
import numpy as np
from sklearn.datasets import load_sample_image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

# 1. ResNet50 모델 로드
model = ResNet50(weights='imagenet')

# 2. sklearn에서 제공하는 샘플 이미지 로드
# 사용 가능한 샘플 이미지: 'china.jpg', 'flower.jpg'
image = load_sample_image('china.jpg')  # 또는 'flower.jpg'
image = img_to_array(image)

# 3. 이미지 전처리
# ResNet50 모델에 입력할 수 있도록 이미지 크기를 224x224로 조정하고, 전처리
image = np.resize(image, (224, 224, 3))
image = np.expand_dims(image, axis=0)
image = preprocess_input(image)

# 4. 예측 수행
predictions = model.predict(image)

# 5. 예측 결과 디코딩 및 출력
decoded_predictions = decode_predictions(predictions, top=3)[0]
print("Predicted:", decoded_predictions)

# 출력된 결과를 해석
for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
    print(f"{i+1}: {label} ({score:.2f})")


102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Predicted: [('n04152593', 'screen', 0.10869243), ('n02840245', 'binder', 0.104571015), ('n06359193', 'web_site', 0.049777426)]
1: screen (0.11)
2: binder (0.10)
3: web_site (0.05)


* screen (0.11):

  모델이 이미지에서 "스크린(screen)"이라고 예측한 확률이 약 11%입니다.
이 예측이 가장 높은 확률로, 모델은 이미지가 스크린과 관련이 있을 가능성이 있다고 판단했습니다.

* binder (0.10):

  모델은 또한 이미지가 "바인더(binder)"일 가능성을 약 10%로 예측했습니다.
이는 바인더(서류철)와 관련된 물체를 인식한 것으로 보입니다.

* web_site (0.05):

  세 번째로, 모델은 이미지가 "웹 사이트(web site)"와 관련이 있을 가능성을 약 5%로 예측했습니다.
  이 결과는 이미지에 웹 페이지와 관련된 요소가 있을 수 있다고 인식한 결과입니다.

In [2]:
import numpy as np
from sklearn.datasets import load_sample_image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# 1. ResNet50 모델 로드 및 미세 조정 설정
base_model = ResNet50(weights='imagenet', include_top=False)  # 최상위 레이어 제거

# 2. 레이어 추가 및 모델 확장
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # 드롭아웃 추가
predictions = Dense(1000, activation='softmax')(x)  # ImageNet 데이터셋의 클래스 수

# 새 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 3. 사전 학습된 레이어 동결
for layer in base_model.layers:
    layer.trainable = False  # ResNet50의 기존 레이어는 학습되지 않도록 동결

# 4. 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 5. 데이터 증강 설정
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 6. 샘플 이미지 로드 및 전처리
image = load_sample_image('china.jpg')  # 예제 이미지
image = img_to_array(image)
image = np.resize(image, (224, 224, 3))  # ResNet50 입력 크기에 맞추기
image = np.expand_dims(image, axis=0)
image = preprocess_input(image)

# 7. 예측 수행
y_pred = model.predict(image)

# 8. 예측 결과 디코딩 및 출력
decoded_predictions = decode_predictions(y_pred, top=3)[0]
print("Predicted:", decoded_predictions)

# 9. 모델 학습 (이 부분은 예제에서 생략됩니다)
# 일반적으로 대규모 데이터셋을 사용하여 학습을 진행해야 합니다.
# datagen.flow(...) 또는 model.fit(...)을 사용하여 데이터 증강된 이미지로 학습 진행 가능

# 조기 종료 설정
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 실제 데이터로 모델 학습
# model.fit(train_data, validation_data=val_data, epochs=50, callbacks=[early_stopping])



94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Predicted: [('n02782093', 'balloon', 0.0087746605), ('n03697007', 'lumbermill', 0.00767095), ('n03478589', 'half_track', 0.0074001565)]


* balloon (0.0088):
모델은 이미지의 일부를 "balloon" (풍선)이라고 인식했을 가능성을 약 0.88%로 예측했습니다.
* lumbermill (0.0077):
모델은 이미지의 일부를 "lumbermill" (제재소)라고 인식할 가능성을 약 0.77%로 예측했습니다.
* half_track (0.0074):
모델은 이미지의 일부를 "half_track" (반트랙, 장갑차의 일종)으로 인식할 가능성을 약 0.74%로 예측했습니다.

In [4]:
import numpy as np
from sklearn.datasets import load_sample_image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# 1. ResNet50 모델 로드 및 미세 조정 설정
base_model = ResNet50(weights='imagenet', include_top=False)  # 최상위 레이어 제거

# 2. 레이어 추가 및 모델 확장
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # 드롭아웃 추가
predictions = Dense(1000, activation='softmax')(x)  # ImageNet 데이터셋의 클래스 수

# 새 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 3. 일부 레이어 동결 해제 (Fine-tuning)
for layer in base_model.layers[-50:]:  # 마지막 50개 레이어는 학습 가능하도록 설정
    layer.trainable = True

# 4. 모델 컴파일 (하이퍼파라미터 튜닝)
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# 5. 데이터 증강 설정
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 데이터의 20%를 검증 데이터로 사용
)

# 6. 샘플 이미지 로드 및 전처리 (여기서는 데이터 증강 예시로 사용)
# 실제로는 여러 이미지를 사용해야 하므로, 아래는 데이터 로딩의 예시입니다.
image = load_sample_image('china.jpg')  # 예제 이미지
image = img_to_array(image)
image = np.resize(image, (224, 224, 3))  # ResNet50 입력 크기에 맞추기
image = np.expand_dims(image, axis=0)
image = preprocess_input(image)

# 7. 데이터 증강을 사용한 학습
# 실제 데이터가 더 많아야 하며, 여기에 추가하여 증강된 이미지를 사용합니다.
train_data, val_data, train_labels, val_labels = train_test_split([image]*100, to_categorical([1]*100, 1000), test_size=0.2)

# Remove the extra dimension from train_data and val_data
train_data = np.squeeze(train_data, axis=1)
val_data = np.squeeze(val_data, axis=1)

train_generator = datagen.flow(
    np.array(train_data),
    np.array(train_labels),
    batch_size=32,
    subset='training'
)

val_generator = datagen.flow(
    np.array(val_data),
    np.array(val_labels),
    batch_size=32,
    subset='validation'
)

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping]
)

# 8. 모델 평가 (검증 데이터에서)
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# 9. 예측 수행 (여기서는 샘플 이미지로 예시)
y_pred = model.predict(image)
decoded_predictions = decode_predictions(y_pred, top=3)[0]
print("Predicted:", decoded_predictions)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 71s 8s/step - accuracy: 0.0000e+00 - loss: 6.5011 - val_accuracy: 0.0000e+00 - val_loss: 6.2487
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step - accuracy: 0.0208 - loss: 6.2271 - val_accuracy: 0.0000e+00 - val_loss: 5.9280
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 247ms/step - accuracy: 0.0729 - loss: 5.9809 - val_accuracy: 0.0000e+00 - val_loss: 5.6887
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.0938 - loss: 5.8532 - val_accuracy: 0.2500 - val_loss: 5.8215
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 241ms/step - accuracy: 0.1250 - loss: 5.5249 - val_accuracy: 0.0000e+00 - val_loss: 5.5861
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step - accuracy: 0.3333 - loss: 5.1115 - val_accuracy: 0.2500 - val_loss: 5.3375
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 256ms/step - accuracy: 0.4688 - loss: 4.9347 - val_accuracy: 0.7500 - val_loss: 5.0371
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.6354 - loss: 4.5631 - val_accuracy: 0.2500 - va

1. ResNet50 모델 로드 및 Fine-tuning:

  ResNet50 모델의 마지막 50개 레이어를 학습 가능하도록 설정하여 더 깊이 Fine-tuning을 진행합니다.

2. 모델 확장 및 드롭아웃 적용:

  GlobalAveragePooling2D, Dense, Dropout 레이어를 추가하여 모델을 확장합니다.
  드롭아웃을 사용하여 과적합을 방지합니다.

3. 데이터 증강 (Data Augmentation):

  다양한 이미지 증강 기법을 적용하여 학습 데이터의 다양성을 증가시킵니다.
  ImageDataGenerator를 사용하여 이미지 데이터를 실시간으로 증강합니다.

4. 모델 컴파일 및 하이퍼파라미터 튜닝:

  학습률(learning rate)을 0.00001로 설정하여 최적화합니다.
  categorical_crossentropy 손실 함수를 사용하여 다중 클래스 분류 작업을 수행합니다.
5. 모델 학습 및 평가:

  학습 과정에서 조기 종료(Early Stopping)를 사용하여 과적합을 방지합니다.
  학습 후 검증 데이터셋에서 모델을 평가하여 성능을 확인합니다.

6. 예측 수행:

  모델을 사용하여 예측을 수행하고, 결과를 디코딩하여 출력합니다.

모델이 성공적으로 학습된 후, 검증 데이터에서 100%의 정확도(accuracy)를 달성한 것을 볼 수 있습니다. 그러나 모델의 val_loss 값이 학습이 진행됨에 따라 다시 증가하는 현상이 나타났습니다. 이는 모델이 검증 데이터에 대해 과적합(overfitting)되고 있을 가능성이 있습니다. 또한, 예측 결과에서 보이는 확률들이 상대적으로 낮은 것을 볼 때, 모델이 완벽히 학습되지 않았을 가능성도 있습니다.

In [5]:
from tensorflow.keras.layers import BatchNormalization

# 2. 레이어 추가 및 모델 확장 (드롭아웃 비율 증가 및 배치 정규화 추가)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)  # 배치 정규화 추가
x = Dropout(0.6)(x)  # 드롭아웃 비율 증가
predictions = Dense(1000, activation='softmax')(x)  # ImageNet 데이터셋의 클래스 수

# 모델 재정의
model = Model(inputs=base_model.input, outputs=predictions)

# 4. 모델 컴파일 (학습률 조정)
model.compile(optimizer=Adam(learning_rate=0.000005), loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정 (patience 감소)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 모델 학습
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping]
)

# 모델 평가 및 예측은 동일
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# 예측 수행 (여기서는 샘플 이미지로 예시)
y_pred = model.predict(image)
decoded_predictions = decode_predictions(y_pred, top=3)[0]
print("Predicted:", decoded_predictions)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.0000e+00 - loss: 8.0329 - val_accuracy: 0.0000e+00 - val_loss: 7.3382
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.0000e+00 - loss: 8.0584 - val_accuracy: 0.0000e+00 - val_loss: 7.4073
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.0000e+00 - loss: 7.8042 - val_accuracy: 0.0000e+00 - val_loss: 7.3833
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 264ms/step - accuracy: 0.0000e+00 - loss: 8.1724 - val_accuracy: 0.0000e+00 - val_loss: 7.2613
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.0000e+00 - loss: 8.0753 - val_accuracy: 0.0000e+00 - val_loss: 7.2982
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.0000e+00 - loss: 8.0669 - val_accuracy: 0.0000e+00 - val_loss: 7.4339
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.0000e+00 - loss: 8.0783 - val_accuracy: 0.0000e+00 - val_loss: 7.2700
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.0000e+00 - l

1. 데이터셋 크기 증가:

  현재 모델은 소량의 데이터로 학습된 것으로 보입니다. 더 많은 데이터를 사용해 모델을 학습시키면 과적합을 줄일 수 있습니다. 더 많은 데이터를 수집하거나, 데이터 증강을 통해 가상 데이터를 생성하는 방법을 사용할 수 있습니다.

2. 드롭아웃(Dropout) 레이어 추가 또는 비율 조정:

  과적합을 줄이기 위해 드롭아웃 레이어의 비율을 높이거나 추가 드롭아웃 레이어를 모델에 추가할 수 있습니다. 현재 설정된 드롭아웃 비율을 0.5에서 0.6이나 0.7로 조정해 볼 수 있습니다.

3. 학습률 조정:

  학습률을 더 낮게 설정하여 모델이 더 천천히, 더욱 정밀하게 학습할 수 있도록 합니다. 이는 과적합을 줄이는 데 도움을 줄 수 있습니다.

4. 얼리 스토핑(Early Stopping) 전략 개선:

  검증 손실(val_loss)을 모니터링하여 더 일찍 학습을 중단할 수 있도록 조기 종료 조건을 더욱 엄격하게 설정할 수 있습니다. patience 매개변수를 줄여 조기 종료를 더 빠르게 할 수 있습니다.

현재의 학습 결과를 보면, 모델의 학습이 제대로 진행되지 않은 것으로 보입니다. 모델의 accuracy와 val_accuracy가 0으로 유지되고 있으며, loss와 val_loss가 거의 변화하지 않고 있습니다. 이는 모델이 데이터를 제대로 학습하지 못하고 있음을 의미합니다.

In [6]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# 모델의 복잡성 줄이기 (Dense 레이어와 뉴런 수 감소)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1000, activation='softmax')(x)

# 새 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 학습률 조정
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping]
)
# 모델 평가 및 예측은 동일
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# 예측 수행 (여기서는 샘플 이미지로 예시)
y_pred = model.predict(image)
decoded_predictions = decode_predictions(y_pred, top=3)[0]
print("Predicted:", decoded_predictions)


Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 65s 7s/step - accuracy: 0.3333 - loss: 4.4525 - val_accuracy: 1.0000 - val_loss: 5.7275e-05
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 1.0000 - loss: 0.0076 - val_accuracy: 1.0000 - val_loss: 5.9628e-05
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step - accuracy: 1.0000 - loss: 5.4734e-06 - val_accuracy: 1.0000 - val_loss: 2.9802e-07
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 247ms/step - accuracy: 1.0000 - loss: 8.6923e-09 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step - accuracy: 1.0000 - l

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Predicted: [('n01443537', 'goldfish', 1.0), ('n04392985', 'tape_player', 1.3625023e-14), ('n02219486', 'ant', 4.856669e-15)]


현재 모델이 매우 높은 정확도와 매우 낮은 손실을 기록하고 있는 것으로 보입니다. accuracy와 val_accuracy가 100%로 나타나고 있으며, loss와 val_loss 값이 거의 0에 수렴하는 상황입니다. 하지만, 이는 일반적으로 모델이 과적합(overfitting)되었거나, 학습 데이터셋이 지나치게 단순해서 발생하는 현상일 수 있습니다.

In [7]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# 모델의 복잡성 줄이기 및 드롭아웃 비율 증가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.6)(x)  # 드롭아웃 비율 증가
predictions = Dense(1000, activation='softmax')(x)

# 새 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 학습률 조정
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 증강 설정
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 데이터의 20%를 검증 데이터로 사용
)

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping]
)
# 모델 평가 및 예측은 동일
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# 예측 수행 (여기서는 샘플 이미지로 예시)
y_pred = model.predict(image)
decoded_predictions = decode_predictions(y_pred, top=3)[0]
print("Predicted:", decoded_predictions)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 58s 4s/step - accuracy: 0.3333 - loss: 5.2474 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 1.0000 - loss: 0.0534 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 1.0000 - loss: 2.4835e-09 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Validation Loss: 0.0, Validation Accuracy: 1.0


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted: [('n01443537', 'goldfish', 1.0), ('n02086646', 'Blenheim_spaniel', 1.1743297e-16), ('n07742313', 'Granny_Smith', 2.1711564e-17)]


과적합 방지 코드

In [ ]:
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
# from tensorflow.keras.optimizers import Adam

# # 모델의 복잡성 줄이기 및 드롭아웃 비율 증가
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)  # 배치 정규화 추가
# x = Dropout(0.7)(x)  # 드롭아웃 비율 증가
# predictions = Dense(10, activation='softmax')(x)  # CIFAR-10 데이터셋의 클래스 수

# # 모델 정의
# model = Model(inputs=base_model.input, outputs=predictions)

# # 학습률 조정
# model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# # 데이터 증강 설정
# datagen = ImageDataGenerator(
#     rotation_range=40,
#     width_shift_range=0.3,
#     height_shift_range=0.3,
#     shear_range=0.2,
#     zoom_range=0.3,
#     horizontal_flip=True,
#     fill_mode='nearest',
#     validation_split=0.2
# )

# # 데이터 증강 적용
# train_generator = datagen.flow(x_train, y_train, batch_size=32)

# # 조기 종료 설정
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # 모델 학습
# model.fit(
#     train_generator,
#     epochs=50,
#     validation_data=(x_test, y_test),
#     callbacks=[early_stopping]
# )

# # 모델 평가 및 예측은 동일
# val_loss, val_accuracy = model.evaluate(val_generator)
# print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# # 예측 수행 (여기서는 샘플 이미지로 예시)
# y_pred = model.predict(image)
# decoded_predictions = decode_predictions(y_pred, top=3)[0]
# print("Predicted:", decoded_predictions)

다른 데이터셋으로 성능을 테스트하는 것은 모델이 다양한 상황에서 얼마나 잘 동작하는지 평가하는 중요한 과정입니다. TensorFlow/Keras는 다양한 공개 데이터셋을 제공합니다. 예를 들어, CIFAR-10, CIFAR-100, 또는 MNIST와 같은 데이터셋을 사용하여 모델의 성능을 테스트할 수 있습니다.

아래는 CIFAR-10 데이터셋을 사용하여 InceptionV3 모델의 성능을 테스트하는 방법을 보여주는 예제입니다. CIFAR-10 데이터셋은 10개의 클래스로 구성된 6만 개의 32x32 크기의 컬러 이미지로 이루어져 있습니다.

CIFAR-10 데이터셋을 사용한 성능 테스트 예제:

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# 1. CIFAR-10 데이터셋 로드 및 전처리
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# InceptionV3 모델에 맞게 이미지 크기를 75x75로 조정
x_train = tf.image.resize(x_train, (75, 75))
x_test = tf.image.resize(x_test, (75, 75))

# 데이터 전처리
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# 레이블을 One-hot 인코딩
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 2. InceptionV3 모델 로드 및 미세 조정 설정
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))

# 3. 레이어 추가 및 모델 확장
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)  # CIFAR-10 데이터셋의 클래스 수

# 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 4. 일부 레이어 동결 해제 (Fine-tuning)
for layer in base_model.layers[-50:]:
    layer.trainable = True

# 5. 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 6. 데이터 증강 설정
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# 데이터 증강 적용
train_generator = datagen.flow(x_train, y_train, batch_size=32)

# 7. 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 8. 모델 학습
model.fit(
    train_generator,
    epochs=50,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]
)

# 9. 모델 평가
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# 10. 예측 수행
y_pred = model.predict(x_test)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 189s 80ms/step - accuracy: 0.4599 - loss: 1.5414 - val_accuracy: 0.8034 - val_loss: 0.5972
Epoch 2/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 86s 55ms/step - accuracy: 0.7589 - loss: 0.7155 - val_accuracy: 0.8470 - val_loss: 0.4502
Epoch 3/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 86s 55ms/step - accuracy: 0.8073 - loss: 0.5821 - val_accuracy: 0.8629 - val_loss: 0.4113
Epoch 4/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 86s 55ms/step - accuracy: 0.8330 - loss: 0.4961 - val_accuracy: 0.8854 - val_loss: 0.3498
Epoch 5/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 87s 55ms/step - accuracy: 0.8491 - loss: 0.4487 - val_accuracy: 0.8791 - val_loss: 0.5642
Epoch 6/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 86s 55ms/step - accuracy: 0.8627 - loss: 0.4096 - val_accuracy: 0.8875 - val_loss: 0.3382
Epoch 7/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 86s 55ms/step - accuracy: 0.8767 - loss: 0.3688 - val_accura